In [9]:
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
import csv
import json
import time
from datetime import datetime

topic_name = "credit_card_trans"
bootstrap_servers = "kafka_v2:9092"
csv_path = "fraudTest.csv" 

In [14]:
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode("utf-8")  
)

In [15]:
import random
def stream_csv_rows(path):
    with open(path, newline='', encoding='utf-8') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                row['event_time'] = datetime.now().isoformat()
                yield row
                delay = random.randint(5, 10)
                time.sleep(delay)

In [16]:
try:
    for i, row in enumerate(stream_csv_rows(csv_path), 1):
        producer.send(topic_name, value=row)
        producer.flush()
        print(f"Produced message {i}: {row}")
except KeyboardInterrupt:
    print("Stopping producer ...\nProducer stopped")
finally:
    producer.flush()
    producer.close()
    #admin_client.close()

Produced message 1: {'': '0', 'trans_date_trans_time': '2020-06-21 12:14:25', 'cc_num': '2291163933867244', 'merchant': 'fraud_Kirlin and Sons', 'category': 'personal_care', 'amt': '2.86', 'first': 'Jeff', 'last': 'Elliott', 'gender': 'M', 'street': '351 Darlene Green', 'city': 'Columbia', 'state': 'SC', 'zip': '29209', 'lat': '33.9659', 'long': '-80.9355', 'city_pop': '333497', 'job': 'Mechanical engineer', 'dob': '1968-03-19', 'trans_num': '2da90c7d74bd46a0caf3777415b3ebd3', 'unix_time': '1371816865', 'merch_lat': '33.986391', 'merch_long': '-81.200714', 'is_fraud': '0', 'event_time': '2025-07-24T13:47:21.697625'}
Produced message 2: {'': '1', 'trans_date_trans_time': '2020-06-21 12:14:33', 'cc_num': '3573030041201292', 'merchant': 'fraud_Sporer-Keebler', 'category': 'personal_care', 'amt': '29.84', 'first': 'Joanne', 'last': 'Williams', 'gender': 'F', 'street': '3638 Marsh Union', 'city': 'Altonah', 'state': 'UT', 'zip': '84002', 'lat': '40.3207', 'long': '-110.436', 'city_pop': '30